In [11]:
import sys
import os
import numpy as np
import pandas as pd
import json
# mcfly
from mcfly import modelgen, find_architecture, storage

In [7]:
data_path = '/media/sf_VBox_Shared/timeseries/actitiracker/WISDM_at_v2.0/'
preprocessed_path = os.path.join(data_path, 'preprocessed')
result_path = os.path.join(data_path, 'models')

In [8]:
X_train = np.load(os.path.join(preprocessed_path, 'X_train.npy'))
X_val = np.load(os.path.join(preprocessed_path, 'X_val.npy'))
X_test = np.load(os.path.join(preprocessed_path, 'X_test.npy'))
y_train = np.load(os.path.join(preprocessed_path, 'y_train.npy'))
y_val = np.load(os.path.join(preprocessed_path, 'y_val.npy'))
y_test = np.load(os.path.join(preprocessed_path, 'y_test.npy'))

In [12]:
with open(os.path.join(preprocessed_path, 'labels.json')) as f:
    labels = json.load(f)

## Generate models

In [9]:
num_classes = y_train.shape[1]

models = modelgen.generate_models(X_train.shape,
                                  number_of_classes=num_classes,
                                  number_of_models = 10)

In [13]:
#what is the fraction of classes in the validation set?
pd.Series(y_val.mean(axis=0), index=labels)

Walking      0.445402
LyingDown    0.055904
Standing     0.082027
Sitting      0.281609
Jogging      0.103971
Stairs       0.031087
dtype: float64

In [14]:
if not os.path.exists(result_path):
        os.makedirs(result_path)

In [15]:
outputfile = os.path.join(result_path, 'modelcomparison.json')
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train,
                                                                           X_val, y_val,
                                                                           models,nr_epochs=5,
                                                                           subset_size=512,
                                                                           verbose=True,
                                                                           batch_size=32,
                                                                           outputfile=outputfile,
                                                                           early_stopping=True)
print('Details of the training process were stored in ',outputfile)

Training model 0 CNN
Train on 512 samples, validate on 3828 samples
Epoch 1/5
512/512 [==============================] - 78s - loss: 4.0337 - acc: 0.3867 - val_loss: 4.2274 - val_acc: 0.2816
Epoch 2/5
512/512 [==============================] - 71s - loss: 3.6176 - acc: 0.5762 - val_loss: 4.0029 - val_acc: 0.2816
Epoch 3/5
512/512 [==============================] - 73s - loss: 3.2745 - acc: 0.6172 - val_loss: 3.7601 - val_acc: 0.2816
Epoch 4/5
512/512 [==============================] - 74s - loss: 2.9327 - acc: 0.6719 - val_loss: 3.5162 - val_acc: 0.2816
Epoch 5/5
512/512 [==============================] - 67s - loss: 2.6737 - acc: 0.7109 - val_loss: 3.3016 - val_acc: 0.2816
Training model 1 CNN
Train on 512 samples, validate on 3828 samples
Epoch 1/5
512/512 [==============================] - 106s - loss: 22.5571 - acc: 0.4121 - val_loss: 19.7033 - val_acc: 0.0820
Epoch 2/5
512/512 [==============================] - 99s - loss: 16.2509 - acc: 0.5566 - val_loss: 13.9189 - val_acc: 0.082